# Lab 8 Supervised Fine Tuning

## 1. Preparing the training data

In [1]:
from datasets import load_dataset

# Load the dataset
# dataset = load_dataset('iamtarun/python_code_instructions_18k_alpaca')
dataset = load_dataset('/share/data/python_code_instructions_18k_alpaca')

dataset = dataset['train']
print(dataset[0])

{'instruction': 'Create a function to calculate the sum of a sequence of integers.', 'input': '[1, 2, 3, 4, 5]', 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'}


In [2]:
# Define a function to transform the data
def format_py18k(example):

    instruction = example['instruction']
    instr_input = example['input']
    output = example['output']

    human_text = f"""{instruction} The given input is {instr_input}."""
    assistant_text = output
    # Apply the new template
    reformatted_entry = {'text': f'<s>[INST] {human_text} [/INST] ```{assistant_text}``` </s>'}

    return reformatted_entry

In [3]:
# Apply the transformation
formatted = dataset.map(format_py18k)
formatted = formatted.remove_columns(['instruction', 'input', 'output', 'prompt'])
formatted = formatted.shuffle(seed=42)

In [4]:
print(formatted[0])

{'text': "<s>[INST] Create a for loop in Python that prints the output of a multiplication table for numbers from 1 to 10. The given input is -. [/INST] ```for i in range(1, 11):\n  for j in range(1, 11):\n    print(i * j, end='\\t')\n  print()``` </s>"}


In [5]:
# Save to your disk
formatted.save_to_disk('/scratch2/py18k')


Saving the dataset (0/1 shards):   0%|          | 0/18612 [00:00<?, ? examples/s]

In [6]:
# (Optional) Push to HF hub (optional)
# transformed_py18k_dataset.push_to_hub("NAME_YOUR_DATASET_ON_HF")